In [31]:
import pandas as pd   
import numpy as np

from bs4 import BeautifulSoup 
import re
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem.porter import PorterStemmer
from keras.datasets import imdb

def clean_review(raw_review):
    #print(raw_review)
    example1 = BeautifulSoup(raw_review, 'html.parser').get_text()
    #print(example1)
    # Use regular expressions to do a find-and-replace
    letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
	                      " ",                   # The pattern to replace it with
	                      example1 )  # The text to search
	# print letters_only
    lower_case = letters_only.lower()        # Convert to lower case
    words = lower_case.split()               # Split into words
	# print words
	# print stopwords.words("english")
    words = [w for w in words if not w in stopwords.words("english")]
	# print words
    from nltk.stem.porter import PorterStemmer
    porter_stemmer = PorterStemmer()
    stemmed_words = []
    for i in range(len(words)):
        stemmed_words.append(porter_stemmer.stem(words[i]))
        return( " ".join( stemmed_words ))


train = pd.read_csv("C:/Users/Ash/Documents/GitHub/nlp_bow/data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
print(train.shape)
# print train.columns.values
num_reviews = train["review"].size
clean_reviews = []
for i in range(num_reviews):
	if( (i+1)%1000 == 0 ):
		print("Review %d of %d\n" % ( i+1, num_reviews ))                                                          
	clean_reviews.append(clean_review(train["review"][i]))


(25000, 3)
Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [21]:
from keras.utils.data_utils import get_file
path = get_file('imdb.npz',origin='https://s3.amazonaws.com/text-datasets/imdb.npz')
f = np.load(path)
x_train = f['x_train']
print(x_train[0])
labels_train = f['y_train']
x_test = f['x_test']
labels_test = f['y_test']
f.close()


[23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215]


In [32]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_reviews[0:24000])

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
print(train_data_features.shape)

Creating the bag of words...

(24000, 1000)


In [20]:
print(x_train[0])

[23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215]


In [33]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"][0:24000] )

Training the random forest...


In [ ]:
# Read the test data
test = pd.read_csv("data/IMDB/testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print test.shape

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
   
    clean_test_reviews.append( clean_review( test["review"][i] ))

In [34]:
clean_test_reviews = clean_reviews[24001:25000]

In [ ]:
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

In [35]:
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)



In [36]:
print(result[0])

1


In [37]:
print(train["sentiment"][24001])

1


In [39]:
print(result==train["sentiment"][24001:25000])

24001     True
24002    False
24003    False
24004     True
24005     True
24006     True
24007    False
24008     True
24009    False
24010    False
24011    False
24012    False
24013    False
24014    False
24015    False
24016    False
24017     True
24018     True
24019     True
24020    False
24021     True
24022     True
24023     True
24024    False
24025    False
24026     True
24027     True
24028     True
24029    False
24030     True
         ...  
24970     True
24971     True
24972    False
24973    False
24974     True
24975    False
24976    False
24977     True
24978     True
24979    False
24980     True
24981    False
24982    False
24983    False
24984    False
24985     True
24986     True
24987     True
24988    False
24989     True
24990    False
24991    False
24992     True
24993     True
24994    False
24995     True
24996     True
24997     True
24998     True
24999     True
Name: sentiment, dtype: bool


In [40]:
c = 1 if result==train["sentiment"][24001:25000] else 0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [42]:
cnt=0
for i in range(0,1000-1):
    if result[i]==train["sentiment"][24001+i]:
        cnt = cnt+1

print(cnt)

581
